In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
db = sqlite3.connect("Monarchs1996.db")

In [3]:
df = pd.read_csv("1996 Season Summary.csv")

        #convert pandas to SQL table
df.to_sql("monarchs1996", db, if_exists="replace")

os.getcwd()

C:\Users\Arink\Anaconda3\lib\site-packages\pandas\core\generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


'C:\\Users\\Arink\\Desktop\\github\\CodeLou\\Monarchs\\Data'

In [4]:
schema = pd.read_sql('SELECT * FROM sqlite_master ORDER by name', db)
schema

,type,name,tbl_name,rootpage,sql
0,index,ix_monarchs1996_index,monarchs1996,4,"CREATE INDEX ""ix_monarchs1996_index""ON ""monarc..."
1,table,monarchs1996,monarchs1996,2,"CREATE TABLE ""monarchs1996"" (\n""index"" INTEGER..."


## Checking DataType

In [5]:
data_type = pd.read_sql('SELECT * FROM PRAGMA_TABLE_INFO("monarchs1996")', db)
data_type

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Tag No.,TEXT,0,None,0
2,2,Mon Sex,TEXT,0,None,0
3,3,Tagger,TEXT,0,None,0
4,4,Where Tagged,TEXT,0,None,0
5,5,Date Tagged,TEXT,0,None,0
6,6,Date Recovered,TEXT,0,None,0
7,7,Where Recovered,TEXT,0,None,0
8,8,Observed or Reported by,TEXT,0,None,0
9,9,Interval,TEXT,0,None,0


### Drop tagger and reporter identification

In [6]:
df.drop(["Tagger", "Mon Sex", "Observed or Reported by", "Interval"], axis = 1, inplace = True)
df

,Tag No.,Where Tagged,Date Tagged,Date Recovered,Where Recovered,Miles
0,IV 301,"Wanamingo,\nMN",09/04/96,03/04/97,"Sierra Chincua,\nMexico",1760.0
1,MC 648,"Mt. Carroll, IL",09/30/96,Feb. 97,"El Rosario,Mexico",1660.0
2,EB 370,"Wamego, KS",09/18/96,Feb. 97,"El Rosario,\nMexico",1375.0
3,MK 276,"Wamego, KS",09/14/96,Feb. 97,"La Herrada,\nMexico",1375.0
4,KN 176,"Wamego, KS",09/12/96,03/10/97,"El Rosario,Mexico",1375.0
...,...,...,...,...,...,...
70,HY 802,"Oklahoma City,\nOK",09/30/96,10/01/96,"Oklahoma City,\nOK",2.0
71,DP 521,"Topeka, KS",10/12/96,10/13/96,"Topeka,KS",2.0
72,DX 480,"Duluth, MN",10/12/96,10/12/96,"Duluth,MN",1.0
73,CT 641,"Dallas, TX",09/24/96,09/24/96,"Dallas,TX",1.0


### Rename columns to match other years

In [7]:
df.rename(columns={'Date Tagged': 'Tag Date', 'Date Recovered': 'Report Date'}, inplace=True)

### Split & rename combined columns in Tagged category

In [8]:
df.loc[df['Where Tagged'].str.contains("\n", na=False), "Where Tagged"] = df.loc[df['Where Tagged'].str.contains("\n", na=False), "Where Tagged"].str.replace("\n", " ")
df2 = df['Where Tagged'].str.split(', ', expand=True, n=1)
df2.columns = ['tag_city', 'Tag State']
df2

,tag_city,Tag State
0,Wanamingo,MN
1,Mt. Carroll,IL
2,Wamego,KS
3,Wamego,KS
4,Wamego,KS
...,...,...
70,Oklahoma City,OK
71,Topeka,KS
72,Duluth,MN
73,Dallas,TX


### Split & Rename combined columns in Reported category

In [9]:
df.loc[df['Where Recovered'].str.contains("\n", na=False), "Where Recovered"] = df.loc[df['Where Recovered'].str.contains("\n", na=False), "Where Recovered"].str.replace("\n", " ")
df3 = df['Where Recovered'].str.split(',', expand=True, n=1)
df3.columns = ['report_city', 'Report State']
df3['Report State'] = df3['Report State'].str.strip()
df3.loc[df3['Report State'] == 'Mexico', 'Report State'] = 'Michoacán'
df3

,report_city,Report State
0,Sierra Chincua,Michoacán
1,El Rosario,Michoacán
2,El Rosario,Michoacán
3,La Herrada,Michoacán
4,El Rosario,Michoacán
...,...,...
70,Oklahoma City,OK
71,Topeka,KS
72,Duluth,MN
73,Dallas,TX


### Create new column Report Country & clean up

In [10]:
add_country = df['Where Recovered'].str.extract("(Mexico)", expand=True)
add_country.columns = ['Report Country']
add_country.loc[add_country['Report Country'] == 'Mexico', 'Report Country'] = 'MEXICO'
add_country.loc[add_country['Report Country'].isnull(), 'Report Country'] = 'USA'
add_country

,Report Country
0,MEXICO
1,MEXICO
2,MEXICO
3,MEXICO
4,MEXICO
...,...
70,USA
71,USA
72,USA
73,USA


### Create new table with unneeded columns dropped

In [11]:
df4 = pd.concat([df, df2, df3, add_country], axis=1)
df4.drop(['Where Tagged', 'Where Recovered', 'report_city', 'tag_city'], axis=1, inplace=True)
df5 = df4[['Tag No.','Tag State', 'Tag Date', 'Report State', 'Report Country', 'Report Date', 'Miles']]
new_table = df5.copy()
new_table

,Tag No.,Tag State,Tag Date,Report State,Report Country,Report Date,Miles
0,IV 301,MN,09/04/96,Michoacán,MEXICO,03/04/97,1760.0
1,MC 648,IL,09/30/96,Michoacán,MEXICO,Feb. 97,1660.0
2,EB 370,KS,09/18/96,Michoacán,MEXICO,Feb. 97,1375.0
3,MK 276,KS,09/14/96,Michoacán,MEXICO,Feb. 97,1375.0
4,KN 176,KS,09/12/96,Michoacán,MEXICO,03/10/97,1375.0
...,...,...,...,...,...,...,...
70,HY 802,OK,09/30/96,OK,USA,10/01/96,2.0
71,DP 521,KS,10/12/96,KS,USA,10/13/96,2.0
72,DX 480,MN,10/12/96,MN,USA,10/12/96,1.0
73,CT 641,TX,09/24/96,TX,USA,09/24/96,1.0


### Final check for unwanted spaces

In [12]:
# new_table.loc[new_table['Report State'].str.contains(r'\s', na=False), 'Report State'].value_counts()
# new_table.loc[new_table['Tag State'].str.contains(r'\s', na=False), 'Tag State'].value_counts()
# new_table.loc[new_table['Tag No.'].str.contains(r'\s', na=False), 'Tag No.'].value_counts()
# new_table.loc[new_table['Report Country'].str.contains(r'\s', na=False), 'Report Country'].value_counts()

### Convert date columns to DateTime datatypes

In [13]:
new_table["Tag Date"]= pd.to_datetime(new_table["Tag Date"]) 
new_table["Report Date"]= pd.to_datetime(new_table["Report Date"]) 
new_table.dtypes

Tag No.                   object
Tag State                 object
Tag Date          datetime64[ns]
Report State              object
Report Country            object
Report Date       datetime64[ns]
Miles                    float64
dtype: object

In [14]:
del df2, df3, df4, df5, add_country

### Export cleaned data to csv.

In [15]:
new_table.to_csv("Monarchs1996.csv")